In [1]:
from ib.ext.Contract import Contract
from ib.ext.Order import Order
from ib.opt import Connection, message
import ib.opt
import ib.ext.Contract
 
import time
import pandas as pd

import sys

if sys.version_info.major == 2:
    import Queue as queue
else:  # >= 3
    import queue

In [47]:
class IbManager(object):
    def __init__(self, timeout=20, **kwargs):
        self.q = queue.Queue()
        self.timeout = 5

        self.con = ib.opt.ibConnection(**kwargs)
        #self.con.registerAll(self.watcher)

        self.msgs = {
            ib.opt.message.error: self.errors,
            ib.opt.message.contractDetails: self.contractDetailsHandler,
            ib.opt.message.contractDetailsEnd: self.contractDetailsHandler
        }

        self.skipmsgs = tuple(self.msgs.keys())

        for msgtype, handler in self.msgs.items():
            self.con.register(handler, msgtype)

        self.con.connect()

    def watcher(self, msg):
        if isinstance(msg, ib.opt.message.error):
            if msg.errorCode > 2000:  # informative message
                print('-' * 10, msg)

        elif not isinstance(msg, self.skipmsgs):
            print('-' * 10, msg)

    def errors(self, msg):
        if msg.id is None:  # something is very wrong in the connection to tws
            self.q.put((True, -1, 'Lost Connection to TWS'))
        elif msg.errorCode < 1000:
            self.q.put((True, msg.errorCode, msg.errorMsg))

    def contractDetailsHandler(self, msg):
        if isinstance(msg, ib.opt.message.contractDetailsEnd):
            self.q.put((False, msg.reqId, msg))
        else:
            self.q.put((False, msg.reqId, msg.contractDetails))

    def get_contract_details(self, contract):
        self.con.reqContractDetails(1, contract)

        cdetails = list()
        df = pd.DataFrame()

        while True:
            try:
                err, mid, msg = self.q.get(block=True, timeout=self.timeout)
            except queue.Empty:
                err, mid, msg = True, -1, "Timeout receiving information"
                break

            if isinstance(msg, ib.opt.message.contractDetailsEnd):
                mid, msg = None, None
                break

            cdetails.append(msg)  # must be contractDetails
            
            if df.shape[0]==0:
                df = pd.DataFrame(columns=list(msg.m_summary.__dict__.keys()))
                df.loc[len(df)] = pd.Series(msg.m_summary.__dict__)
            else:
                df.loc[len(df)] = pd.Series(msg.m_summary.__dict__)

        return df, err, mid, msg

In [44]:
def vixFutContract():
    contract = Contract()
    contract.m_symbol = "VIX"
    contract.m_exchange = "CFE"
    contract.m_secType = "FUT"
    contract.m_tradingClass = 'VX'

    return contract

def esFutContract():
    contract = Contract()
    contract.m_symbol = "ES"
    contract.m_exchange = "GLOBEX"
    contract.m_secType = "FUT"
    contract.m_tradingClass = 'ES'
    
    return contract

def esOptContract(exp):
    contract = Contract()
    contract.m_symbol = "ES"
    contract.m_secType = "FOP"
    contract.m_exchange = "GLOBEX"
    #contract.m_right = "C"
    #contract.m_localSymbol = spFuts[0].m_localSymbol
    contract.m_expiry = exp
 
    return contract

In [48]:
ibm.con.disconnect()
ibm = IbManager("127.0.0.1", port=7496, clientId=999)

Server Version: 76
TWS Time at connection:20181218 11:31:17 EST


In [49]:
def vxOptContract(exp):
    contract = Contract()
    contract.m_symbol = "VIX"
    contract.m_secType = "OPT"
    contract.m_exchange = "SMART"
    #contract.m_right = "C"
    #contract.m_localSymbol = spFuts[0].m_localSymbol
    contract.m_expiry = exp
    contract.m_tradingClass='VIX'
 
    return contract

In [50]:
#cdetails, err, errid, errmsg = ibm.get_contract_details(vixFutContract())
#cdetails, err, errid, errmsg = ibm.get_contract_details(esFutContract())
#cdetails, err, errid, errmsg = ibm.get_contract_details(esOptContract(cdetails.loc[0].m_expiry))
cdetails, err, errid, errmsg = ibm.get_contract_details(vxOptContract('20190115'))
cdetails

AttributeError: 'str' object has no attribute 'm_summary'

In [42]:
for msg in cdetails:
    print msg.m_summary.m_expiry,msg.m_summary.m_tradingClass,msg.m_summary.m_localSymbol,msg.m_summary.m_strike

 20190115 VIX VIX   190116C00010000 10.0
20190115 VIX VIX   190116C00011000 11.0
20190115 VIX VIX   190116C00012000 12.0
20190115 VIX VIX   190116C00013000 13.0
20190115 VIX VIX   190116C00014000 14.0
20190115 VIX VIX   190116C00015000 15.0
20190115 VIX VIX   190116C00016000 16.0
20190115 VIX VIX   190116C00017000 17.0
20190115 VIX VIX   190116C00018000 18.0
20190115 VIX VIX   190116C00019000 19.0
20190115 VIX VIX   190116C00020000 20.0
20190115 VIX VIX   190116C00021000 21.0
20190115 VIX VIX   190116C00022000 22.0
20190115 VIX VIX   190116C00023000 23.0
20190115 VIX VIX   190116C00024000 24.0
20190115 VIX VIX   190116C00025000 25.0
20190115 VIX VIX   190116C00026000 26.0
20190115 VIX VIX   190116C00027000 27.0
20190115 VIX VIX   190116C00028000 28.0
20190115 VIX VIX   190116C00029000 29.0
20190115 VIX VIX   190116C00030000 30.0
20190115 VIX VIX   190116C00032500 32.5
20190115 VIX VIX   190116C00035000 35.0
20190115 VIX VIX   190116C00037500 37.5
20190115 VIX VIX   190116C00040000 40.0